In [22]:
import pandas as pd
import requests
import pandas as pd
import plotly.express as px
from collections import Counter
import plotly.io as pio

In [23]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../EveryTraceOfFampayPeriod/filtered_links.csv')

# Initialize an empty DataFrame to store the results
results = pd.DataFrame(columns=['Link', 'Cache-Control', 'Expires', 'Content-Encoding'])

In [24]:
# Iterate over the links in the DataFrame
for link in df['Link']:
    try:
        # Make a request to the link
        response = requests.get(link)

        # Extract the headers related to caching and compression
        cache_control = response.headers.get('Cache-Control')
        expires = response.headers.get('Expires')
        content_encoding = response.headers.get('Content-Encoding')

        # Append the results to the DataFrame
        results = results.append({
            'Link': link,
            'Cache-Control': cache_control,
            'Expires': expires,
            'Content-Encoding': content_encoding
        }, ignore_index=True) # type: ignore

        results.to_csv('cacheandcontent.csv', index=False)

    except Exception as e:
        print(f"Error fetching link {link}: {e}")

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_26732\2254892318.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_26732\2254892318.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_26732\2254892318.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_26732\2254892318.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\Harsh Clean\AppData\Local\Temp\ipykernel_26732\2254892318.py:13: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a

In [25]:
# Load the CSV file into a DataFrame
data = pd.read_csv('cacheandcontent.csv')
data.head()

,Link,Cache-Control,Expires,Content-Encoding
0,https://famapp.in/,NaN,NaN,gzip
1,https://famcard.me/,NaN,NaN,gzip
2,https://fampay.in/,NaN,NaN,gzip
3,https://fampay.in/about,NaN,NaN,gzip
4,https://fampay.in/blog,max-age=604800,"Mon, 29 May 2023 05:29:41 GMT",gzip


In [26]:
# Count the occurrence of each encoding
content_encoding_counts = data['Content-Encoding'].value_counts()

# Convert the Series to a DataFrame
content_encoding_counts_df = content_encoding_counts.reset_index()

# Rename the columns for the plot
content_encoding_counts_df.columns = ['Content-Encoding', 'Count']

# Create the bar plot
fig = px.bar(content_encoding_counts_df, x='Content-Encoding', y='Count', title='Distribution of Content-Encoding Values')

# Display the plot
fig.show()
pio.write_html(fig, file='../../fam-report-site/public/Web-Analytics/contentencoding.html', auto_open=True)

In [27]:
# Count the occurrence of each Cache-Control value
cache_control_counts = data['Cache-Control'].value_counts()

# Convert the Series to a DataFrame
cache_control_counts_df = cache_control_counts.reset_index()

# Rename the columns for the plot
cache_control_counts_df.columns = ['Cache-Control', 'Count']

# Create the pie chart
fig = px.pie(cache_control_counts_df, names='Cache-Control', values='Count', title='Distribution of Cache-Control Values')

# Display the plot
fig.show()
pio.write_html(fig, file='../../fam-report-site/public/Web-Analytics/disturbutioncachecontrol.html', auto_open=True)

In [28]:
# Convert the 'Expires' column to datetime
data['Expires'] = pd.to_datetime(data['Expires'], errors='coerce')

# Create the histogram
fig = px.histogram(data, x='Expires', nbins=50, title='Distribution of Expires Values')

# Display the plot
fig.show()
pio.write_html(fig, file='../../fam-report-site/public/Web-Analytics/disturbutionexpires.html', auto_open=True)

In [29]:
# Initialize a Counter
cache_control_counter = Counter()

# Update the counter for each record
for cache_control in data['Cache-Control'].dropna():
    cache_control_counter.update(cache_control.split(','))

# Convert the Counter to a DataFrame
cache_control_counts_df = pd.DataFrame.from_records(list(cache_control_counter.items()), columns=['Cache-Control', 'Count'])

# Create the treemap
fig = px.treemap(cache_control_counts_df, path=['Cache-Control'], values='Count', title='Treemap of Cache-Control Directives')

# Display the plot
fig.show()
pio.write_html(fig, file='../../fam-report-site/public/Web-Analytics/treemapcachecontrol.html', auto_open=True)

In [34]:
# Replace NaN values in 'Content-Encoding' and 'Cache-Control' with 'Not specified'
data['Content-Encoding'] = data['Content-Encoding'].fillna('Not specified')
data['Cache-Control'] = data['Cache-Control'].fillna('Not specified')

# Create a new column that combines 'Content-Encoding' and 'Cache-Control'
data['Combined'] = data['Content-Encoding'] + ' - ' + data['Cache-Control']

# Now create the sunburst chart
fig = px.sunburst(data, path=['Combined'], title='Sunburst Chart of Cache-Control and Content-Encoding Combined')

# Display the plot
fig.show()
pio.write_html(fig, file='../../fam-report-site/public/Web-Analytics/sunburstcachecontrol.html', auto_open=True)